In [25]:
import tflearn
from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.conv import conv_2d, max_pool_2d
from tflearn.layers.normalization import local_response_normalization
from tflearn.layers.estimator import regression
from tflearn.metrics import Accuracy
from utils import mnist_reader
import numpy as np
import tensorflow as tf

%matplotlib inline

In [2]:
X_train, y_train = mnist_reader.load_mnist('data/fashion', kind='train')
X_test, y_test = mnist_reader.load_mnist('data/fashion', kind='t10k')

In [3]:
X_train = X_train.reshape([-1 , 28 , 28 , 1])

In [4]:
X_test = X_test.reshape([-1 , 28 , 28 , 1])

In [5]:
acc = Accuracy()

In [6]:
# Building 'AlexNet'
network = input_data(shape=[None, 28, 28, 1])
network = conv_2d(network, 96, 11, strides=4, activation='relu')
network = max_pool_2d(network, 3, strides=2)
network = local_response_normalization(network)
network = conv_2d(network, 256, 5, activation='relu')
network = max_pool_2d(network, 3, strides=2)
network = local_response_normalization(network)
network = conv_2d(network, 384, 3, activation='relu')
network = conv_2d(network, 384, 3, activation='relu')
network = conv_2d(network, 256, 3, activation='relu')
network = max_pool_2d(network, 3, strides=2)
network = local_response_normalization(network)
network = fully_connected(network, 4096, activation='tanh')
network = dropout(network, 0.5)
network = fully_connected(network, 4096, activation='tanh')
network = dropout(network, 0.5)
network = fully_connected(network, 10, activation='softmax')
network = regression(network, metric=acc , optimizer='momentum',
                     loss='categorical_crossentropy',
                     to_one_hot = True , 
                     n_classes = 10 , 
                     learning_rate=0.001)

In [7]:
model = tflearn.DNN(network, checkpoint_path='./models/fashion_alexnet',
                    max_checkpoints=1, tensorboard_verbose=2 , tensorboard_dir = "./logs/")

In [9]:
model.fit(X_train, y_train, n_epoch=10, validation_set=(X_test , y_test), shuffle=True,
          show_metric=True, batch_size=40, snapshot_step=200,
          snapshot_epoch=False, run_id='alexnet_oxflowers17')

Training Step: 14999  | total loss: 0.30188 | time: 1208.473s
| Momentum | epoch: 010 | loss: 0.30188 - acc: 0.9243 -- iter: 59960/60000
Training Step: 15000  | total loss: 0.29109 | time: 1210.912s
| Momentum | epoch: 010 | loss: 0.29109 - acc: 0.9219 | val_loss: 0.34266 - val_acc: 0.8787 -- iter: 60000/60000
--


In [14]:
model.save("./models/fashion_alexnet.tflearn")

In [15]:
model_alexnet = model

In [37]:
tf.reset_default_graph()
# Building 'VGG Network'
graph = tf.Graph()
with graph.as_default():
    network = input_data(shape=[None, 28 , 28 , 1])

    network = conv_2d(network, 64, 3, activation='relu')
    network = conv_2d(network, 64, 3, activation='relu')
    network = max_pool_2d(network, 2, strides=2)

    network = conv_2d(network, 128, 3, activation='relu')
    network = conv_2d(network, 128, 3, activation='relu')
    network = max_pool_2d(network, 2, strides=2)

    network = conv_2d(network, 256, 3, activation='relu')
    network = conv_2d(network, 256, 3, activation='relu')
    network = conv_2d(network, 256, 3, activation='relu')
    network = max_pool_2d(network, 2, strides=2)

    network = conv_2d(network, 512, 3, activation='relu')
    network = conv_2d(network, 512, 3, activation='relu')
    network = conv_2d(network, 512, 3, activation='relu')
    network = max_pool_2d(network, 2, strides=2)

    network = conv_2d(network, 512, 3, activation='relu')
    network = conv_2d(network, 512, 3, activation='relu')
    network = conv_2d(network, 512, 3, activation='relu')
    network = max_pool_2d(network, 2, strides=2)

    network = fully_connected(network, 4096, activation='relu')
    network = dropout(network, 0.5)
    network = fully_connected(network, 4096, activation='relu')
    network = dropout(network, 0.5)
    network = fully_connected(network, 10, activation='softmax')

    network = regression(network, optimizer='rmsprop',
                         loss='categorical_crossentropy',
                         to_one_hot = True , 
                         n_classes = 10 , 
                         learning_rate=0.0001)

In [29]:
model2 = tflearn.DNN(network, checkpoint_path='./models/model_vgg',
                    max_checkpoints=1, tensorboard_verbose=2 , tensorboard_dir = "./logs/")

In [19]:
y_pred = model_alexnet.predict(X_test[:8000])

In [20]:
sum(np.argmax(y_pred , axis=1) == y_test[:8000])/8000.0

0.877

In [21]:
model2 = tflearn.DNN(network, checkpoint_path='./models/model_vgg',
                    max_checkpoints=1, tensorboard_verbose=2 , tensorboard_dir = "./logs/")

In [38]:
import cPickle as pickle

In [40]:
pickle.dump(y_pred , open("y_pred.dat" , "wb") , True)